## Type Inference and Generic Methods

* _type inference_: the Java compiler's ability to determine the argument's type for a method invocation based on its corresponding declaration
    - the inference algorithm also determines the type that the result is being assigned or returned
    - it tries to find the most specific type that works with all of the arguments
    - __note: the inference algorithm uses only invocation arguments, target types, and maybe an expected return type to infer types. it does not use results from later in the program__
* in the example below:
    - inference determines that the second argument passed to the pick method is of type Serializable
    - the first argument is of type String and implements the Serializable interface
    - the second argument also implements the Serializable interface

In [ ]:
static <T> T pick(T a1, T a2) { return a2; }
Serializable s = pick("d", new ArrayList<String>());

* generic methods also have type inference and enable you to invoke a generic method just like any ordinary method without specifying a type between angle brackets
* in the example below:
    - addBox() defines 1 type parameter named U
    - we can choose to call this method by specifying the type argument in the angle brackets
    - or go with the diamond notation and have the compiler infer the type based on the method declaration and type of the arguments

In [ ]:
public class BoxDemo {

  public static <U> void addBox(U u, 
      java.util.List<Box<U>> boxes) {
    Box<U> box = new Box<>();
    box.set(u);
    boxes.add(box);
  }

  public static <U> void outputBoxes(java.util.List<Box<U>> boxes) {
    int counter = 0;
    for (Box<U> box: boxes) {
      U boxContents = box.get();
      System.out.println("Box #" + counter + " contains [" +
             boxContents.toString() + "]");
      counter++;
    }
  }

  public static void main(String[] args) {
    java.util.ArrayList<Box<Integer>> listOfIntegerBoxes =
      new java.util.ArrayList<>();
    BoxDemo.<Integer>addBox(Integer.valueOf(10), listOfIntegerBoxes);
    BoxDemo.addBox(Integer.valueOf(20), listOfIntegerBoxes);
    BoxDemo.addBox(Integer.valueOf(30), listOfIntegerBoxes);
    BoxDemo.outputBoxes(listOfIntegerBoxes);
  }
}

// can specify the type parameter with a type witness
BoxDemo.<Integer>addBox(Integer.valueOf(10), listOfIntegerBoxes);

// or can omit it and the Java compiler can infer the argument type
// based on the method's type parameters
BoxDemo.addBox(Integer.valueOf(20), listOfIntegerBoxes);

## Type Inference and Instantiation of Generic Classes

* can replace type arguments required to invoke the constructor of a generic class with an empty set of type parameters (\<\>) as long as the compiler can infer the type arguments from the context
    - this is called diamond notation

In [ ]:
// specifying the type arguments
Map<String, List<String>> myMap = new HashMap<String, List<String>>();

// using diamond notation
Map<String, List<String>> myMap = new HashMap<>();

// caused an unchecked conversion warning
// you must use diamond notation to take advantage of type inference
// in this case, the HashMap() constructor refers to the HashMap raw type
// and not the Map<String, List<String>> type
Map<String, List<String>> myMap = new HashMap(); // unchecked conversion warning

## Type Inference and Generic Constructors of Generic and Non-Generic Classes

* constructors can be generic, i.e. declare their own formal type parameters in both generic and non-generic classes
* in the example below:
    - we create an instance of the parameterized type MyClass<Integer>
        * this specifies the type Integer for the formal type parameter X of the generic class MyClass<X>
    - the constructor for this generic class contains a formal type parameter, T
        * the compiler infers that the type for T is String based on the type of the argument passed in

In [ ]:
class MyClass<X> {
  <T> MyClass(T t) {
    // ...
  }
}

// prior to Java SE 7, the compiler is able to infer the type
// parameters of generic constructors, similar to generic methods

new MyClass<Integer>("")

// in Java SE 7+, compilers can also infer the type parameters
// of the generic class being instantiated if one uses the diamond notation (<>)

MyClass<Integer> myObject = new MyClass<>("");

## Target Types

* the compiler takes advantage of target typing to infer the type parameters of a generic method invocation
* __target type__: of an expression is the data type that the Java compiler expects depending on where the expression appears
    - e.g. you call a generic method that returns an Object<T>, where T is the generic type
        * if you assign the result of that method to a variable, like String result = genericMethod(), then the target type would be String
    - some examples of target types:
        * variable declarations
        * assignments
        * return statements
        * array initializers
        * method or constructor arguments
        * lambda expression bodies
        * conditional expressions, ?: (ternary operator)
        * cast expressions
* consider the method Collections.emptyList() which is declared below:

In [ ]:
static <T> List<T> emptyList();

* in the example below, the statement is expecting an instance of List\<String\>
    - this is the target type
* because emptyList() returns a value of type List<T>, the compiler can infer that the type argument, T, is a String
    - you could also just a type witness and specify the value of T for the emptyList() method

In [ ]:
List<String> listOne = Collections.emptyList();

// using a type witness
// this is not necessary since the compiler can infer
// that T in <T> is a String
List<String> listOne = Collections.<String>emptyList();

* however, using a type witness would be necessary in the example below:
    - the compiler requires a value for the type argument, T, so it starts with Object
    - in doing so, Collections,emptyList() returns a value of List<Object>
        * this is incompatible with processStringList()

In [ ]:
void processStringList(List<String> stringList) {
    // process stringList
}

// this does not compile
// generates an error message:
// List<Object> cannot be converted to List<String>
processStringList(Collections.emptyList());

// in Java SE 7, you must specify the value of the type argument
// b/c it cannot infer the type here
processStringList(Collections.<String>emptyList());

* in Java SE 8, the notion of what is a target type is expanded to include _method arguments_ such as the argument to the method processStringList()
    - since processStringList() requires an argument of type List<String>
    - the method Collections.emptyList() returns a value of List<T> and using the expanded definition of target type, we can say that the target type is List<String>
    - therefore, type argument T has a value of String

In [ ]:
// in Java SE 8, this compiles
// the target type takes into account the type of the method arguments
// processStringList takes in List<String> types
// .emptyList() returns a List<T> type
// therefore, the compiler infers that T is of type String

processStringList(Collections.emptyList());

## Target Typing in Lambda Expressions

* suppose you have the following methods, how would you determine the type of the lambda expression in these cases?
    - the lambda expression in both methods is the same
* when the Java runtime invokes the method printPersons():
    - it expects a data type of CheckPerson
    - therefore, the lambda expression is of this type
* when the Java runtime invokes the method printPersonsWithPredicate():
    - it is expecting a data type of Predicate<Person>
    - so the lambda expression is of this type
* _the data type that these methods expect is called the target type_
    - the Java compiler uses the target type to determine the lambda expressions' type
    - the target type is based on the context or situation in which the lambda expression was found
    - __therefore, you can only use lambda expressions in situations in which the Java compiler can determine a target type__
        * variable declarations
        * assignments
        * return statements
        * array initializers
        * method or constructor arguments
        * lambda expression bodies
        * conditional expressions, ?: (ternary operator)
        * cast expressions

In [ ]:
public static void printPersons(List<Person> roster, CheckPerson tester)

public void printPersonsWithPredicate(List<Person> roster, Predicate<Person> tester) 

printPersons(
        people, 
        p -> p.getGender() == Person.Sex.MALE
            && p.getAge() >= 18
            && p.getAge() <= 25);)

printPersonsWithPredicate(
        people,
        p -> p.getGender() == Person.Sex.MALE
             && p.getAge() >= 18
             && p.getAge() <= 25);)

## Target Types and Method Arguments

* for method arguments, the Java compiler determines the target type with two other language features:
    1. overload resolution
    2. type argument inference
* consider the following:
    - Runnable.run() does not return a value
    - Callable<V>.call() does
    - we have 2 overloaded methods, invoke() that take in arguments of type Runnable and Callable<T>
    - if we have a statement, String s = invoke(() -> "done"), which of the 2 overloaded methods would be called?
        * the compiler sees that invoke should return a value of type String
        * the only method that would return anything is the one that takes in type Callable<T>
        * therefore, the lambda expression, () -> "done", is of type Callable<T>

In [ ]:
public interface Runnable {
    void run();
}

public interface Callable<V> {
    V call();
}

// we have an overloaded method, invoke, as follows
// same method name but different signatures
void invoke(Runnable r) {
    r.run();
}

<T> T invoke(Callable<T> c) {
    return c.call();
}

// which invoke method will be invoked?
// in this case, the second one
// reason being, invoke is supposed to return something
// Runnable.r() does not return anything but Callable<T> does
// therefore, the type of the lambda expression, () -> "done" is Callable<T>

String s = invoke(() -> "done");